`BpForms` is a toolkit for unambiguously describing the primary sequence of biopolymers such as DNA, RNA, and proteins, including modified DNA, RNA, and proteins. BpForms represents biopolymers as monomeric forms linked.
This tutorial illustrates how to use the `BpForms` Python API. Please see the [documentation](https://docs.karrlab.org/bpforms/) for more information about the `BpForms` grammar and more instructions for using the `BpForms` website, JSON REST API, and command line interface.

### Import `BpForms`

In [1]:
import bpforms

### Create instances of `BpForm`
Use the [`BpForm` grammar](https://docs.karrlab.org/bpforms) and the `BpForm.from_str` method to create an instance of `BpForm` to represent a form of a biopolymer.

`BpForms` includes six predefined alphabet and six corresponding pre-defined subclasses of `BpForm`:
* Canonical DNA nucleotide monophosphates (`bpforms.canonical_dna_alphabet`, `bpforms.CanonicalDnaForm`): canonical nucleotide monophosphates
* Canonical RNA nucleotide monophosphates (`bpforms.canonical_rna_alphabet`, `bpforms.CanonicalRnaForm`): canonical nucleotide monophosphates
* Canonical protein amino acids (`bpforms.canonical_protein_alphabet`, `bpforms.CanonicalProteinForm`): canonical protein amino acids
* DNA nucleotide monophosphates (`bpforms.dna_alphabet`, `bpforms.DnaForm`): canonical nucleotide monophosphates plus non-canonical nucleotide monophosphates based on [DNAmod](https://dnamod.hoffmanlab.org)
* RNA nucleotide monophosphates (`bpforms.rna_alphabet`, `bpforms.RnaForm`): canonical nucleotide monophosphates plus non-canonical nucleotide monophosphates based on [MODOMICS](http://modomics.genesilico.pl/modifications/) and the [RNA Modification Database](https://mods.rna.albany.edu/mods/)
* Protein amino acids (`bpforms.protein_alphabet`, `bpforms.ProteinForm`): canonical protein amino acids plus non-canonical protein amino acids based on the [PDB Chemical Component Dictionary](http://www.wwpdb.org/data/ccd) and [RESID](https://pir.georgetown.edu/resid/)

#### Create `BpForm`s composed canonical monomeric forms
Monomeric forms defined in the alphabets can be referenced by their single character codes.

In [2]:
dna_form = bpforms.DnaForm().from_str('ACGT')

#### Create `BpForm`s that includes non-canonical monomeric forms
Some of the non-canonical monomeric forms in the alphabets are represented by multiple characters. Their character codes must be delimited by curly brackets.

In [3]:
dna_form = bpforms.DnaForm().from_str('A{m2C}GT')

#### Create `BpForm`s that include monomeric forms that are not defined in the alphabet
Additional monomeric forms can be described in square brackets using one or more attributes separated by vertical pipes ("|"):
* `id`
* `name`
* `synonym`
* `identifier`
* `structure`: canonical SMILES-encoded string that represents the structure of the monomeric form
* `r-bond-atom`: list of atoms involved in bond(s) with monomeric forms to the left
* `r-displaced-atom`: list of atoms displaced by the formation of bond(s) with monomeric forms to the left
* `l-bond-atom`: list of atoms involved in bond(s) with monomeric forms to the right
* `l-displaced-atom`: list of atoms displaced by the formation of bond(s) with monomeric forms to the right
* `delta-mass`: additional mass in Daltons beyond that described by the `structure` attribute; used to represent uncertainty in the structure of the monomeric form.
* `delta-charge`: additional charge beyond that described by the `structure` attribute; used to represent uncertainty in the structure of the monomeric form.
* `position`: represents uncertainty in the location of a non-canonical monomeric form.
* `base-monomer`: represents the parent monomeric form of the monomeric form (e.g. the parent of m2A is A)
* `comments`

In [4]:
dna_form = bpforms.DnaForm().from_str(
    '''A[
         id: "m2C" 
         | name: "2-O-methylcytosine"
         | synonym: "4-amino-2-methoxypyrimidine"
         | synonym: "o-2-methylcytosine"
         | identifier: "CHEBI:70854" @ "chebi"
         | structure: "COC1=NC(=CCN1C1CC(C(O1)COP(=O)([O-])[O-])O)N"
         | r-bond-atom: O20
         | l-bond-atom: P16
         | r-displaced-atom: H20
         | l-displaced-atom: O19-1
         | comments: "Methylation of deoxycytidine"
         | base-monomer: "C"
         | position: 2-3
        ]GT'''.replace('\n', '').replace(' ', ''))

### Get and set monomeric forms and slices of monomeric forms of biopolymers
Individual residues and slices of residues can be get and set similar to lists.

In [5]:
dna_form[0]

In [6]:
dna_form[2] = bpforms.dna_alphabet.monomers.A

In [7]:
dna_form[2:4]

In [8]:
dna_form[2:4] = bpforms.DnaForm().from_str('TA')

### Get and set the bases of monomeric forms

In [9]:
dna_form[1].base_monomers

{<bpforms.core.Monomer at 0x7f2d4d086940>}

### Calculate the major protonation state of biopolymer forms at specific pHs

In [10]:
microspecies = dna_form.get_major_micro_species(8.)
microspecies.GetTotalCharge()

-5

In [11]:
microspecies = dna_form.get_major_micro_species(3.)
microspecies.GetTotalCharge()

-2

### Calculate physical properties of biopolymer forms
`BpForms` can calculate the length, formula, molecular weight, and charge of the biopolymer forms.

In [12]:
len(dna_form)

4

In [13]:
str(dna_form.get_formula())

'C40H49N15O24P4'

In [14]:
dna_form.get_mol_wt()

1247.808047992

In [15]:
dna_form.get_charge()

-5

### Get IUPAC/IUBMB representations of biopolymer forms
`BpForms` can generate IUPAC/IUBMB representations of forms. This is useful for understanding the semantic meaning of a form using tools such as BLAST. Where annotated, this method uses the `base_monomers` attribute to represent modified monomeric forms using the codes for their roots (e.g. m2A is represented as "A"). Monomeric forms without annotated bases are represented as "N".

In [16]:
dna_form.get_canonical_seq()

'ACTA'

In [17]:
dna_form_2 = bpforms.DnaForm().from_str(
    '''A[
         id: "m2C" 
         | name: "2-O-methylcytosine"
         | synonym: "4-amino-2-methoxypyrimidine"
         | synonym: "o-2-methylcytosine"
         | identifier: "CHEBI:70854" @ "chebi"
         | structure: "COC1=NC(=CCN1C1CC(C(O1)COP(=O)([O-])[O-])O)N"
         | r-bond-atom: O20
         | l-bond-atom: P16
         | r-displaced-atom: H20
         | l-displaced-atom: O19-1
         | comments: "Methylation of deoxycytidine"
         | base-monomer: "C"
         | position: 2-3
        ]GT'''.replace('\n', '').replace(' ', ''))
dna_form_2.get_canonical_seq()

'ACGT'

In [18]:
dna_form_2 = bpforms.DnaForm().from_str(
    '''A[
         id: "m2C" 
         | name: "2-O-methylcytosine"
         | synonym: "4-amino-2-methoxypyrimidine"
         | synonym: "o-2-methylcytosine"
         | identifier: "CHEBI:70854" @ "chebi"
         | structure: "COC1=NC(=CCN1C1CC(C(O1)COP(=O)([O-])[O-])O)N"
         | r-bond-atom: O20
         | l-bond-atom: P16
         | r-displaced-atom: H20
         | l-displaced-atom: O19-1
         | comments: "Methylation of deoxycytidine"
         | position: 2-3
        ]GT'''.replace('\n', '').replace(' ', ''))
dna_form_2.get_canonical_seq()

'ANGT'

### Determine if biopolymers are equal
Use the `is_equal` method to check if two biopolymers are equal.

In [19]:
dna_form_1 = bpforms.DnaForm().from_str('ACGT')
dna_form_2 = bpforms.DnaForm().from_str('ACGT')
dna_form_3 = bpforms.DnaForm().from_str('GCTC')

In [20]:
dna_form_1.is_equal(dna_form_2)

True

In [21]:
dna_form_1.is_equal(dna_form_3)

False